Sentiment Example from SLP Section 5.1.1

In [2]:
import numpy as np

After feature extraction our example review can be boiled down to a vector of 6 features. And we know that its correct class is positive (1). Let's hardcode the example and the class.

In [4]:
example = np.array([3,2,1,3,0,4.15])

In [3]:
correct = 1

Our initial weights and bias, as given in the example.

In [5]:
weights = np.array([2.5, -5, -1.2, 0.5, 2.0, 0.7])

In [6]:
bias = .1

We score our example with the usual method, a dot product of the weight vector and feature vector. 

In [7]:
rawscore = np.dot(weights, example) + bias

In [8]:
rawscore

0.80500000000000027

And then turn that into a proper probability with the sigmoid.

In [9]:
def classprob(score): return 1/(1+np.exp(-score))

In [10]:
score = classprob(rawscore)

In [11]:
score

0.69104301241572286

Now that we have the probability assigned to the example we can compute the gradient (partial derivative with respect to each feature). 

Here we're using numpy's array capabilities more fully. Instead of writing a loop where we multiply the (score - correct) by each feature value, we'll multiply 'example' directly; the result of this multiply is an array where each element of example is multiplied by (score - correct). 

In [12]:
gradient = (score - correct) * example

In [57]:
gradient

array([-0.92687096, -0.61791398, -0.30895699, -0.92687096, -0.        ,
       -1.2821715 ])

With the gradient in hand, we can update the weights. Assuming a learning rate of 1 just to make things consistent with the slides.

In [13]:
learningrate = 1

In [14]:
new_weights = weights - learningrate * gradient

In [15]:
new_weights

array([ 3.42687096, -4.38208602, -0.89104301,  1.42687096,  2.        ,
        1.9821715 ])

Now that we have the new weights we can reassess the class probability of our example.

In [16]:
new_score = classprob(np.dot(new_weights, example) + bias)

And not surprisingly with only example and a big learning rate, we make a big leap towards the right answer (not realistic).

In [65]:
new_score

0.99999820772397829